In [33]:
import re

In [36]:
starters = re.findall("\-\- cte\n,\s+[0-9a-zA-Z_]+\s+AS\s+\(",query)
ctes = re.split("\-\- cte\n,\s+[0-9a-zA-Z_]+\s+AS\s+\(",query)[1:]
#for i in range(len(ctes)):
i = 0
cte_i = ctes[i]
cte_i

'\nSELECT\nSS0.* FROM street_segment SS0\n)\n\n'

In [40]:
starters[0]

'-- cte\n, street_segment_orig AS ('

In [18]:
starters

['-- cte\n, landuse_orig AS (',
 '-- cte\n, traffic_volume_orig AS (',
 '-- cte\n, street_segment_projected AS (',
 '-- cte\n, street_segment_cleaned_0 AS (',
 '-- cte\n, street_segment_cleaned AS (',
 '-- cte\n, street_segment_buffers AS (',
 '-- cte\n, landuse_projected AS (',
 '-- cte\n, landuse_cleaned AS (',
 '-- cte\n, street_segment_buffer_landuses AS (',
 '-- cte\n, street_segment_buffer_landuses_pivoted AS (',
 '-- cte\n, street_segment_geoenriched AS (',
 '-- cte\n, traffic_volume_cleaned AS (',
 '-- cte\n, traffic_volume_agg_directions AS (',
 '-- cte\n, traffic_volume_agg_time AS (',
 '-- cte\n, final_data AS (']

In [3]:
ctes = (query.split("-- cte")[1:-1])

def save_cte_to_sql(cte_i):
    split_by_from = cte_i.split("FROM")
    before_from = split_by_from[0]
    after_from = split_by_from[1]
    source_table_name_extractor = after_from.strip(" ").strip("\n").split(" ")
    source_table_name = source_table_name_extractor[0]
    source_table_alias = source_table_name_extractor[1].replace("\n","").replace(")","")
    source_table_name_ref = "ref( {{'"+source_table_name+"'}})"
    new_table_name = before_from.lower().replace(","," ").replace("\n","").strip(" ").split("as")[0].strip(" ")
    new_table_alias = "".join([w[0].upper() for w in new_table_name.split("_")])+"0"

    cte_i2sql = "{{ config(materialized='table') }}" + "\n\n"
    cte_i2sql += "WITH" + "\n"
    cte_i2sql += before_from[2:]
    cte_i2sql += " FROM " + source_table_name_ref + " " + source_table_alias + ")" + "\n"
    cte_i2sql += "SELECT " + new_table_alias + ".* " + " FROM " + new_table_name + " " + new_table_alias

    with open("{0}.sql".format(new_table_name), "w") as f:
        f.write(cte_i2sql)
        f.close()
    return cte_i2sql


for cte_i in ctes:
    save_cte_to_sql(cte_i)

In [ ]:
street_segment_cleaned_query = """
{{ config(materialized='table') }}

{% set projection = var('projection') %}
{% set null_placeholder = var('null_placeholder') %}

WITH
street_segment_orig AS (
SELECT
SS0.* FROM street_segment SS0
)

-- cte
, street_segment_projected AS (
SELECT
SS0.*,
ST_Transform(SS0.geometry, {{ projection }}) as geom
FROM street_segment_orig SS0
)

-- cte
, street_segment_cleaned_0 AS (
SS0.Segment_ID
, SS0.SHAPE_Leng AS SHAPE_Leng
, SS0.StreetWidt::DOUBLE AS StreetWidt
, SS0.Number_Tra::INTEGER AS Number_Tra
, COALESCE(SS0.FeatureTyp, {{ null_placeholder }}) AS FeatureTyp
, COALESCE(SS0.RW_TYPE, {{ null_placeholder }}) AS RW_TYPE
, COALESCE(SS0.TrafDir, {{ null_placeholder }}) AS TrafDir
, COALESCE(SS0.Status, {{ null_placeholder }}) AS Status
, SS0.geom as geometry
FROM street_segment_projected SS0
)

-- cte
, street_segment_cleaned AS (
SSC0.Segment_ID
, SSC0.SHAPE_Leng
, SSC0.StreetWidt
, SSC0.Number_Tra
, SSC0.FeatureTyp
, SSC0.RW_TYPE
, SSC0.TrafDir
, SSC0.Status
, SSC0.geometry
FROM street_segment_cleaned_0 SSC0
WHERE
SSC0.Segment_ID NOT NULL
AND SSC0.StreetWidt >= 8.0
AND SSC0.Number_Tra >= 1
AND SSC0.RW_Type IN (['1','2','3','4','9','11','13',{{ null_placeholder }})
AND SSC0.FeatureType NOT IN ('F', 'A', 'W', '9', '8','2','5')
AND (SSC0.TrafDir <> 'P' OR SSC0.TrafDir <> {{ null_placeholder }})
AND (SSC0.Status = '2' OR SSC0.Status = {{ null_placeholder }})
)
SELECT SSC1.* FROM street_segment_cleaned SSC1
"""

landuse_cleaned_query = """
{{ config(materialized='table') }}


{% set projection = var('projection') %}


WITH
landuse_orig AS (
SELECT
L0.* FROM landuse L0
)

-- cte
, landuse_projected AS (
SELECT
L0.*,
ST_Transform(L0.geometry,{{projection}}) as geom
FROM landuse_orig L0
)

-- cte
, landuse_cleaned AS (
SELECT
L0.geom as geometry
, L0.LandUse
FROM landuse_projected L0
)
SELECT LC0.* FROM landuse_cleaned LC0
"""


street_segment_buffers_query = """
{{ config(materialized='table') }}


{% set radius = var('radius') %}


WITH 
street_segment_buffers AS (
SELECT
SSC1.Segment_ID,
ST_BUFFER(SSC1.geometry, {{radius}}) AS geometry
FROM ref( {{'street_segment_cleaned'}}) SSC1
)
SELECT SSB0.* FROM street_segment_buffers
"""

street_segment_buffer_landuses_query = """
{{ config(materialized='table') }}


WITH
street_segment_buffer_landuses AS (
SELECT
SSB0.Segment_ID,
LC0.LandUse,
FROM ref( {{'street_segment_buffers'}}) AS SSB0
JOIN ref( {{'landuse_cleaned'}} ) AS LC0
ON ST_Contains(LC0.geometry, SSB0.geometry) OR ST_Intersects(LC0.geometry, SSB0.geometry)
)
SELECT SSBL0.* FROM street_segment_buffer_landuses SSBL0
"""

street_segment_geoenriched_query = """
{{ config(materialized='table') }}


WITH
street_segment_buffer_landuses_pivoted AS (
SELECT
SSBL0.Segment_ID
, SUM(CASE WHEN (SSBL0.LandUse = '01') THEN 1 ELSE 0 END) AS LandUse_01
, SUM(CASE WHEN (SSBL0.LandUse = '02') THEN 1 ELSE 0 END) AS LandUse_02
, SUM(CASE WHEN (SSBL0.LandUse = '03') THEN 1 ELSE 0 END) AS LandUse_03
, SUM(CASE WHEN (SSBL0.LandUse = '04') THEN 1 ELSE 0 END) AS LandUse_04
, SUM(CASE WHEN (SSBL0.LandUse = '05') THEN 1 ELSE 0 END) AS LandUse_05
, SUM(CASE WHEN (SSBL0.LandUse = '06') THEN 1 ELSE 0 END) AS LandUse_06
, SUM(CASE WHEN (SSBL0.LandUse = '07') THEN 1 ELSE 0 END) AS LandUse_07
, SUM(CASE WHEN (SSBL0.LandUse = '08') THEN 1 ELSE 0 END) AS LandUse_08
, SUM(CASE WHEN (SSBL0.LandUse = '09') THEN 1 ELSE 0 END) AS LandUse_09
, SUM(CASE WHEN (SSBL0.LandUse = '10') THEN 1 ELSE 0 END) AS LandUse_10
, SUM(CASE WHEN (SSBL0.LandUse = '11') THEN 1 ELSE 0 END) AS LandUse_11
, SUM(CASE WHEN ((SSBL0.LandUse IS NULL) OR (SSBL0.LandUse NOT IN ('01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11'))) THEN 1 ELSE 0 END) AS LandUse_NULL
COUNT(*) AS tempNumLandUses
FROM ref({{'street_segment_buffer_landuses'}}) SSBL0
GROUP BY SSBL0.Segment_ID
)


-- cte
, street_segment_geoenriched AS (
SELECT
SS1.Segment_ID
, SS1.StreetWidt
, SS1.Number_Tra
, SS1.SHAPE_Leng
, SSBLP0.LandUse_01/SSBLP0.tempNumLandUses AS LandUse_01
, SSBLP0.LandUse_02/SSBLP0.tempNumLandUses AS LandUse_02
, SSBLP0.LandUse_03/SSBLP0.tempNumLandUses AS LandUse_03
, SSBLP0.LandUse_04/SSBLP0.tempNumLandUses AS LandUse_04
, SSBLP0.LandUse_05/SSBLP0.tempNumLandUses AS LandUse_05
, SSBLP0.LandUse_06/SSBLP0.tempNumLandUses AS LandUse_06
, SSBLP0.LandUse_07/SSBLP0.tempNumLandUses AS LandUse_07
, SSBLP0.LandUse_08/SSBLP0.tempNumLandUses AS LandUse_08
, SSBLP0.LandUse_09/SSBLP0.tempNumLandUses AS LandUse_09
, SSBLP0.LandUse_10/SSBLP0.tempNumLandUses AS LandUse_10
, SSBLP0.LandUse_11/SSBLP0.tempNumLandUses AS LandUse_11
, SSBLP0.LandUse_NULL/SSBLP0.tempNumLandUses AS LandUse_NULL
FROM ref({{'street_segment_cleaned'}}) SS1
LEFT JOIN
street_segment_buffer_landuses_pivoted SSBLP0
ON
SS1.Segment_ID = SSBLP0.Segment_ID
)
SELECT SSG0.* FROM street_segment_geoenriched SSG0
"""

traffic_volume_cleaned_query = """
{{ config(materialized='table') }}


WITH
traffic_volume_orig AS (
SELECT
TV0.* FROM traffic_volume TV0
)

-- cte
, traffic_volume_cleaned AS (
SELECT
TV0.*
, TV0.M // 3 AS Season
, EXTRACT(ISODOW FROM MAKE_DATE(TV0.Yr, TV0.M, TV0.D)) AS DayOfWeek
, (CASE WHEN (EXTRACT(ISODOW FROM MAKE_DATE(TV0.Yr, TV0.M, TV0.D)) < 6) THEN 0 ELSE 1 END) AS IsWeekend
FROM traffic_volume_orig TV0
)
SELECT TVC0.* FROM traffic_volume_cleaned TVC0
"""

traffic_volume_agg_time_query = """
{{ config(materialized='table') }}


WITH
traffic_volume_agg_directions AS (
SELECT
TV0.SegmentID
, TV0.Yr
, TV0.M
, TV0.D
, TV0.HH
, TV0.MM
, SUM(TV0.Vol) AS Vol
FROM ref({{'traffic_volume_cleaned'}}) TV0
GROUP BY
TV0.SegmentID
, TV0.Yr
, TV0.M
, TV0.D
, TV0.HH
, TV0.MM
)

-- cte
, traffic_volume_agg_time AS (
SELECT
TV0.SegmentID
, TV0.Season
, TV0.IsWeekend
, TV0.HH
, AVG(TV0.Vol) AS Vol
FROM traffic_volume_agg_directions TV0
GROUP BY
TV0.SegmentID
, TV0.Season
, TV0.IsWeekend
, TV0.HH
)
SELECT TVAT0.* FROM traffic_volume_agg_time TVAT0
"""

final_data_query = """
{{ config(materialized='table') }}


WITH
final_data AS (
SELECT
SSGE0.*,
TVAT0.Season,
TVAT0.IsWeekend,
TVAT0.HH AS HH,
FROM ref({{'traffic_volume_agg_time'}}) TVAT0
LEFT JOIN ref({{'street_segment_geoenriched'}}) SSGE0
ON TVT0.SegmentID = SSGE0.Segment_ID
)
SELECT FD0.* FROM final_data FD
"""

In [2]:
[str(x).zfill(2) for x in range(1,12)]

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']